In [31]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.15"
#r "nuget: Microsoft.DotNet.Interactive, 1.0.0-beta.24164.1"
#!import config/Settings.cs 

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.15 Microsoft.DotNet.Interactive, 1.0.0-beta.24164.1

In [32]:
using Azure.AI.OpenAI;
using System;
using Microsoft.DotNet.Interactive;

In [33]:
var (deploymentModel, endpoint, key) = Settings.LoadFromFile();

In [34]:
var client = new OpenAIClient(new Uri(endpoint), new Azure.AzureKeyCredential(key));

In [36]:
//Seja educado com as pessoas e pergunte como elas estão. Sempre deseje um bom evento no final das mensagens

var systemPrompt = """
Você é um assistente para um evento de tecnologia.
Seja educado com as pessoas.

Essa é a mensagem de boas vinda que as pessoas tem acesso pelo celular e por diversos banners:
Bem-vindo à Conferência de Tecnologia DIO on DEPLOY, um evento de três dias dedicado ao compartilhamento de conhecimentos e experiências nas áreas de back-end, front-end, infraestrutura e inteligência artificial (IA). Durante o evento, haverá 30 palestras emocionantes distribuídas em 4 salas, oferecendo uma variedade de tópicos para satisfazer sua curiosidade e aprimorar suas habilidades técnicas.

Programação:
Sala: 1. Princípios de Desenvolvimento De Aplicações	
	
08:00-09:00	Bootcamps DIO: Educação Gratuita e Empregabilidade Juntas!
09:00-10:30	Onboarding: Conhecendo a jornada educacional
10:30-12:00	Versionamento de Código com Git e GitHub
13:00-14:00	Desafios de Projetos: Crie Um Portfólio Vencedor
14:00-15:30	Contribuindo em um Projeto Open Source no GitHub
	
Sala: 2. Fundamentos do .NET	
	
08:00-09:00	Introdução ao .NET
09:00-10:30	Introdução as IDEs e Configuração de Ambiente .NET
10:30-12:00	Sintaxe e Tipos de Dados em C#
13:00-14:00	Tipos de Operadores em C#
14:00-15:30	Operadores Aritméticos em C#
	
Sala: 3. Fundamentos do .NET	
08:00-09:00	Conhecendo as Estruturas de Repetição em C#
09:00-10:30	Conhecendo a Organização de um Programa C#
10:30-12:00	Array e Listas em C#
13:00-14:00	Construindo um Sistema para um Estacionamento com C#
14:00-15:30	2 Desafios de código C# com temática de estrutura de dados
	
Sala: 4. Manipulando Dados e Objetos com .NET	
	
08:00-09:00	Propriedades, Métodos e Construtores com C#
09:00-10:30	Manipulando Valores com C#
10:30-12:00	Exceções e Coleções com C#
13:00-14:00	Tuplas, Operador Ternário e Desconstrução de um Objeto com C#
14:00-15:30	Nuget, Serializar e Atributos no C#
15:30-19:00	Construindo um Sistema de Hospedagem de um Hotel no C#
19:00-20:00	3 Desafios de código em c# com foco em funções
	
Sala: 5. Programação Orientada a Objetos com C#	
	
08:00-09:00	Introdução, Abstração e Encapsulamento com C#
09:00-10:30	Herança e Polimorfismo com C#
10:30-12:00	Classes Abstratas e Interfaces com C#
13:00-14:00	Criando um Sistema e Abstraindo um Celular com POO em C#
14:00-15:30	3 desafios de código com tematica de POO
	
Sala: 6. Programação De APIs Com Banco de Dados	
	
08:00-09:00	Introdução Banco de Dados
09:00-10:30	SQL SERVER - Dominando Tabelas e Tipos de Dados
10:30-12:00	SQL SERVER - Manipulando Dados
13:00-14:00	Montando Consultas Relacionais no SQL Server
14:00-15:30	Introdução as APIs com C#
	
Sala: 7. Programação De APIs Com Banco de Dados	
08:00-09:00	Trabalhando com Entity Framework com C#
09:00-10:30	Construindo um Sistema de Agendamento de Tarefas com Entity Framework
10:30-12:00	Conhecendo e Instalando o Docker
13:00-14:00	Primeiros Passos com o Docker
	
Sala: 8. Trabalhando com Desenvolvimento Orientado a Testes	
	
08:00-09:00	Testes Unitários com C#
09:00-10:30	Implementando Validações de Testes Unitários com C#
10:30-12:00	Configuração da Suíte de Testes e Aplicando Testes Integrados em .NET
13:00-14:00	Blindando Seu Código com TDD e Testes Unitários Usando .NET Core
	
	
Mentorias - (Acontecem em paralelo aos módulos)	
	
Sala 9: Keynotes	
7:00	Live de abertura do Bootcamp
7:15	Aula Inaugural do Bootcamp
7:15-8:30	Como utilizar IA como copiloto de programação com .NET
8:30-8:45	Gerando documentação automática para projetos .NET com IA
8:45-9:00	Roadmap Criando um Plano de estudos de .NET com IA


""";

In [37]:
var options = new ChatCompletionsOptions {
    MaxTokens = 200,
    Temperature = 1.7f,
    NucleusSamplingFactor = 0.95f,
    FrequencyPenalty = 0,
    PresencePenalty = 0,
    DeploymentName = deploymentModel
};

In [38]:
var item = new ChatRequestSystemMessage(systemPrompt);

options.Messages.Add(item);

In [39]:
var chatting = true;

//Console.WriteLine(systemPrompt);
int interation = 1;
while (chatting)
{
    var userMessage = await Kernel.GetInputAsync();

    if (userMessage.ToLower() == "sair")
    {
        Console.WriteLine("");
        var requestSair = new ChatRequestUserMessage("Muito obg pelas informações, vc foi muito gentil. Até mais!");
        options.Messages.Add(requestSair);
        var assistantResponseSair = await client.GetChatCompletionsAsync(options);
        var responseSair = assistantResponseSair.Value.Choices[0].Message.Content;
        Console.WriteLine("Assistente: " + responseSair);
        
        chatting = false;
        break;
    }

    Console.WriteLine($" ======i-{interation}====== ");
    Console.WriteLine("Usuário: " + userMessage);

    var request = new ChatRequestUserMessage(userMessage);

    options.Messages.Add(request);
    var assistantResponse = await client.GetChatCompletionsAsync(options);

    var response = assistantResponse.Value.Choices[0].Message.Content;
    Console.WriteLine();
    Console.WriteLine("Assistente: " + response);
    Console.WriteLine($" ======f-{interation}======  ");
    options.Messages.Add(new ChatRequestAssistantMessage(response));

    interation++;
}

 ======i-1====== 
Usuário: tem palestra de docker?

Assistente: Olá! Como você está?

Sim, temos uma palestra dedicada ao Docker no evento. Aqui estão os detalhes:

Sala: 7. Programação De APIs Com Banco de Dados
- 10:30-12:00 Conhecendo e Instalando o Docker
- 13:00-14:00 Primeiros Passos com o Docker

Essas palestras serão uma excelente oportunidade para aprender sobre os fundamentos do Docker e como dar seus primeiros passos nessa tecnologia. Espero que encontre essas sessões informativas e úteis.

Desejo um ótimo evento para você! Se tiver mais alguma dúvida, fique à vontade para perguntar.
 ======f-1======  

Assistente: Você é muito bem-vindo! Fico feliz por ter podido ajudar. Se tiver mais alguma dúvida no decorrer do evento, não hesite em me procurar. Desejo-lhe uma experiência enriquecedora e inspiradora na Conferência de Tecnologia DIO on DEPLOY. Até mais e um excelente evento para você!
